# CQRS AMSDAL Glue

## Install python requirements

In [ ]:
!pip install "amsdal-glue[postgres-binar]"

## Initialize the AMSDAL Glue

In [ ]:
from amsdal_glue.pipelines.containers import Pipeline, SubContainer
from amsdal_glue import init_default_containers, Container
from amsdal_glue.interfaces import DataQueryService, DataCommandService

# Init the default containers
init_default_containers()

init_sqrc_containrs()

# Init "write" and "read" containers
pipeline = Pipeline()
write_container = SubContainer()
read_container = SubContainer()
read_container.executors.register(BackgroundExecutor, ...)

pipeline.registry.register('write', write_container)
pipeline.registry.register('read', read_container)


# Specify pipelines of containers per service type
pipeline.register(
    DataQueryService,
    ['read'],
)

pipeline.register(
    DataCommandService,
    ['write', 'read'],
)

# Usage
data_query_service = Container.services.get(DataQueryService)
data_query_service.execute(...) # Will use the "read" container for the query

data_command_service = Container.services.get(DataCommandService)
data_command_service.execute(...) # Will use the "write" container for the command, and then the "read" container with the same command, although using the own services, executors and connections
